Test PyBamm

In [1]:
import pybamm as pb

model = pb.lithium_ion.DFN()
sim = pb.Simulation(model)
sim.solve([0, 3600])

Model variables and outputs

In [2]:
#model.variable_names()
#model.variables.search("electrolyte")

#output_variables = ["Electrolyte concentration [mol.m-3]", "Voltage [V]"] #må også oppgi enkeltplot i lister!
#sim.plot(output_variables = output_variables)

#nesting lists
'''
sim.plot(
    [
        ["Electrode current density [A.m-2]", "Electrolyte current density [A.m-2]"],
        "Voltage [V]",
    ]
)
'''

#contributions of overpotential
#sim.plot_voltage_components()
#dette kan også bli splittet i elektrodene ved å skrive 
#sim.plot_voltage_components(split_by_electrode=True)

Electrolyte concentration [Molar]
Electrolyte concentration [mol.m-3]
Electrolyte concentration concatenation [mol.m-3]
Electrolyte convection flux [mol.m-2.s-1]
Electrolyte current density [A.m-2]
Electrolyte diffusion flux [mol.m-2.s-1]
Electrolyte flux [mol.m-2.s-1]
Electrolyte migration flux [mol.m-2.s-1]
Electrolyte potential [V]
Electrolyte transport efficiency
Gradient of electrolyte potential [V.m-1]
Gradient of negative electrolyte potential [V.m-1]
Gradient of positive electrolyte potential [V.m-1]
Gradient of separator electrolyte potential [V.m-1]
Loss of lithium inventory, including electrolyte [%]
Negative electrolyte concentration [Molar]
Negative electrolyte concentration [mol.m-3]
Negative electrolyte potential [V]
Negative electrolyte transport efficiency
Positive electrolyte concentration [Molar]
Positive electrolyte concentration [mol.m-3]
Positive electrolyte potential [V]
Positive electrolyte transport efficiency
Separator electrolyte concentration [Molar]
Separat

'\nsim.plot(\n    [\n        ["Electrode current density [A.m-2]", "Electrolyte current density [A.m-2]"],\n        "Voltage [V]",\n    ]\n)\n'

Parameter values

In [4]:
parameter_values = pb.ParameterValues("Chen2020")
#specific parameter values can be accessed using standard dictionary syntax: 
parameter_values["Electrode height [m]"]
#can also search parameter values with 
#parameter_values.search("electrolyte")

#check the specific parameters the model requires and what kind of parameter it is: 
#model.print_parameter_info()
# a function parameter can always be defined to be a constant, but a parameter cannot be defined to be a function. 
#change constant parameters 
parameter_values["Current function [A]"] = 10


Defining parameters

In [4]:
import numpy as np
def my_current(t):
    return pb.sin(2 * np.pi * t / 60)


parameter_values["Current function [A]"] = my_current
#the actual name given to the argument does not matter, but the order does. Check print_parameter_info. 

sim = pb.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)
sim.solve(t_eval=t_eval)
sim.plot(["Current [A]", "Voltage [V]"]) 

#we can also define a new parameter set from scratch (like Amund has done).
#see tutorial 4 on pybamm for more information, under "define a new parameter set". 
#it is often convenient to define the parameter set in a separate file, and then call  
#import by get_parameter_values method. 



interactive(children=(FloatSlider(value=0.0, description='t', max=120.0, step=1.2), Output()), _dom_classes=('…

Experiments and cycles

In [5]:
#we can reproduce real cycling conditions via tuples - see Tutorial 5. 
#key difference between cycles and steps; steps can be skipped, but not cycles.
# model = pybamm.lithium_ion.DFN()
#sim = pybamm.Simulation(model, experiment=experiment) 

#we can choose to plot a specific cycle: 
#sim.solution.cycles[0].plot()

#we can pass additional arguments to a step (periods etc)
#pybamm.step.string("Discharge at 1C for 1 hour", period="1 minute", temperature="25oC", tags=["tag1"])
#can also directly program instead of using string formatting: 
#pybamm.step.current(1, duration="1 hour", termination="2.5 V")
#can also use drive cycles; see Tutorial 5. 

Post-processed variables

In [6]:
solution = sim.solve([0, 3600])
#defining post-processed variable for the relevant variables: 
V = solution["Voltage [V]"]
#these processedvariable objects contain the datapoints for the corresponding variable
#can be accessed by calling the entries variable. 
V.entries
#post-processed variables can be called at any time, which will return the interpolated value from the data above
#V([200, 400, 780, 1236])  # times in seconds

array([4.18095279, 4.2008079 , 4.13529144, 4.22601343, 4.15002952,
       4.17825622, 4.19999   ])

Saving simulation and output data

In [7]:
#sim.save("DFNe.pkl")
#we can load the stored simulation doing 
#sim2 = pb.load("DFNe.pkl")
#we can also just save the solution of the simulation
#sol = sim.solution
#sol.save("SPMe_sol.pkl")
#can also just save the data for some variables, also in .csv or .mat format, or matlab (see tutorial 6)
#sol.save_data("sol_data.pkl", ["Current [A]", "Voltage [V]"], to_format = "csv")

#remove data
#import os
#os.remove("DFNe.pkl")


Solver options and tolerances

In [8]:
#Might want to tighten the tolerances to obtain a more accurate solution
#or might want to loosen the tolerances to reduce the solve time. 

#Casadisolver
#you can integrate until you hit a certain voltage - reccommended for simulations of a full charge or discharge. 

Mesh

In [9]:
#default number of mesh points used in a simulation
#you may find you need to increase the number points in the mesh to obtain an accurate solution. 
#good practice to conduct a mesh refinement study --> simulate the same problem with a finer mesh and compare the results. 

#look at the default numbers
#model.default_var_pts

#we can create our own dictionary (see tutorial 9)
#we can do a mesh refinement study (see tutorial 9)
#can use labels in the dynamic plot method so the plots are labeled. 